# LLM JP instruction-tuning Finetuning with HuggingFace and Weights and Biases
<!--- @wandbcode{llm-finetune-hf} -->
- Fine-tune a lightweight LLM with LoRA and 8-bit quantization
- Checkpoint the LoRA adapter weights as artifacts

# Setting

In [1]:
import bitsandbytes as bnb
import copy
import glob
import os
import wandb
import json
from tqdm import tqdm
from types import SimpleNamespace
import datasets
from datasets import Dataset
import random
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, Trainer, TrainingArguments
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from peft import PeftModel, PeftConfig, LoraConfig, get_peft_model


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["WANDB_ENTITY"] = "japan-demo"
os.environ["WANDB_PROJECT"] = "jp-instruction-tuning"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
os.environ["WANDB_WATCH"] = "gradients"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb=1024"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
wandb.login()

wandb: Currently logged in as: keisuke-kamata (wandb). Use `wandb login --relogin` to force relogin


True

## Install Model and Dataset

In [3]:
config = {
    "BASE_MODEL":"cyberagent/open-calm-3b",
    "lora_config":{
        "r":32,
        "lora_alpha":16,
        "target_modules":["query_key_value"],
        "lora_dropout":.1,
        "bias":"none",
        "task_type":"CAUSAL_LM"
    },
    "training_args":{
        "dataloader_num_workers":16,
        "evaluation_strategy":"steps",
        "per_device_train_batch_size":8,
        "max_steps": 100,
        "gradient_accumulation_steps":2,
        "report_to":"wandb",#wandb integration
        "warmup_steps":10,
        "num_train_epochs":1,
        "learning_rate":2e-4,
        "fp16":True,
        "logging_steps":10,
        "save_steps":25,
        "output_dir":'./outputs'
    }
}

In [4]:
torch.cuda.empty_cache()
model = AutoModelForCausalLM.from_pretrained(
    config["BASE_MODEL"],
    load_in_8bit=True,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(config["BASE_MODEL"])

In [5]:
dolly_ja = datasets.load_dataset("kunishou/databricks-dolly-15k-ja")
dolly_ja = list(dolly_ja['train'])
split_index = int(len(dolly_ja) * 0.8)  # 例として80%をトレーニングデータとして使用
dolly_ja_train = dolly_ja[:split_index]
dolly_ja_val = dolly_ja[split_index:]

## Dataset

In [6]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task. Write a response that appropriately completes the request."
        "### Instruction:{instruction} \n\n Input:{input} \n\n ###Response"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. Write a response that appropriately completes the request."
        "### Instruction:{instruction} \n\n ###Response"
    )
}

class InstructDataset(Dataset):
    def __init__(self, json_list, tokenizer, ignore_index=-100):
        self.tokenizer = tokenizer
        self.ignore_index = ignore_index
        self.features = []
        
        for j in tqdm(json_list):
            if 'input' in j:
                source_text = PROMPT_DICT['prompt_input'].format_map(j)
            else:
                source_text = PROMPT_DICT['prompt_no_input'].format_map(j)
            example_text = source_text + j['output'] + self.tokenizer.eos_token
            
            source_tokenized = self.tokenizer(
                source_text,
                padding='longest',
                truncation=True,
                max_length=512,
                return_length=True,
                return_tensors='pt'
            )
            
            example_tokenized = self.tokenizer(
                example_text, 
                padding='longest', 
                truncation=True, 
                max_length=512, 
                return_tensors='pt'
            )
            
            input_ids = example_tokenized['input_ids'][0]
            labels = copy.deepcopy(input_ids)
            source_len = source_tokenized['length'][0]
            labels[:source_len] = self.ignore_index
            
            self.features.append({
                'input_ids': input_ids,
                'labels': labels
            })
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx]


class InstructCollator():
    def __init__(self, tokenizer, ignore_index=-100):
        self.tokenizer = tokenizer
        self.ignore_index = -100

    def __call__(self, examples):
        input_batch = []
        label_batch = []
        for example in examples:
            input_batch.append(example['input_ids'])
            label_batch.append(example['labels'])
        input_ids = pad_sequence(
            input_batch, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = pad_sequence(
            label_batch, batch_first=True, padding_value=self.ignore_index
        )
        attention_mask = input_ids.ne(self.tokenizer.pad_token_id)   
        return {
            'input_ids': input_ids,
            'labels': labels,
            'attention_mask': attention_mask
        }

In [7]:
train_dataset = InstructDataset(dolly_ja_train, tokenizer)
val_dataset = InstructDataset(dolly_ja_val, tokenizer)
collator = InstructCollator(tokenizer)

100%|██████████| 3003/3003 [00:01<00:00, 1823.92it/s]


## Post processing on the model

In [ ]:
# モデルの中身を確認
print(model.gpt_neox.layers[0].attention)
#GPTNeoXAttention(
#  (rotary_emb): RotaryEmbedding()
#  (query_key_value): Linear(in_features=4096, out_features=12288, bias=True)
#  (dense): Linear(in_features=4096, out_features=4096, bias=True)

# cast the small parameters (e.g. layernorm) to fp32 for stability
for param in model.parameters():
    param.requires_grad = False # freeze the model - train adapters later
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)
model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()
class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
model.embed_out = CastOutputToFloat(model.embed_out)

## Training

In [ ]:
with wandb.init(config=config, job_type="training") as run:
    # Setup for LoRa
    lora_config = LoraConfig(**wandb.config["lora_config"])
    model_peft = get_peft_model(model, lora_config)    
    model_peft.print_trainable_parameters()
    model_peft.config.use_cache = False

    trainer = transformers.Trainer(
        model=model_peft,
        data_collator=collator,
        args=transformers.TrainingArguments(**wandb.config["training_args"]),
        train_dataset=train_dataset,
        eval_dataset=val_dataset
    )
    trainer.train()
    model_peft.save_pretrained("./output")
    model_ft = wandb.Artifact(f"finetuned-model", type="model")
    model_ft.add_dir("./output")
    run.log_artifact(model_ft)
    run.log_code()

## (Advanced) Sweep

In [8]:
sweep_configuration= {
    "method": "random",
    "metric": {"goal": "minimize", "name": "eval/loss"},
    "parameters": {
        "r":{"values": [2,4,8,16,32]},
        "lora_alpha":{"values": [2,4,8,16]},
        "learning_rate":{'max': 2e-3, 'min': 2e-4}
    }
}

default_config = {
    "BASE_MODEL":"cyberagent/open-calm-3b",
    "lora_config":{
        "r":32,
        "lora_alpha":16,
        "target_modules":["query_key_value"],
        "lora_dropout":.1,
        "bias":"none",
        "task_type":"CAUSAL_LM"
    },
    "training_args":{
        "dataloader_num_workers":16,
        "evaluation_strategy":"steps",
        "per_device_train_batch_size":8,
        "max_steps": 100,
        "gradient_accumulation_steps":2,
        "report_to":"wandb",#wandb integration
        "warmup_steps":10,
        "num_train_epochs":1,
        "learning_rate":2e-4,
        "fp16":True,
        "logging_steps":10,
        "save_steps":25,
        "output_dir":'./outputs'
    }
}

# cast the small parameters (e.g. layernorm) to fp32 for stability
for param in model.parameters():
    param.requires_grad = False # freeze the model - train adapters later
    if param.ndim == 1:
        param.data = param.data.to(torch.float32)
model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()
class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
model.embed_out = CastOutputToFloat(model.embed_out)


def train_func():
    with wandb.init(config=config, job_type="training") as run:
        # Setup for LoRa
        default_config["lora_config"]["r"] = wandb.config["r"]
        default_config["lora_config"]["lora_alpha"] = wandb.config["lora_alpha"]
        default_config["training_args"]["learning_rate"] = wandb.config["learning_rate"]


        lora_config = LoraConfig(**default_config["lora_config"])
        model_peft = get_peft_model(model, lora_config)    
        model_peft.print_trainable_parameters()
        model_peft.config.use_cache = False
    
        trainer = transformers.Trainer(
            model=model_peft,
            data_collator=collator,
            args=transformers.TrainingArguments(**default_config["training_args"]),
            train_dataset=train_dataset,
            eval_dataset=val_dataset
        )
        trainer.train()
        model_peft.save_pretrained("./output")
        model_ft = wandb.Artifact(f"finetuned-model", type="model")
        model_ft.add_dir("./output")
        run.log_artifact(model_ft)
        run.log_code()

sweep_id = wandb.sweep(sweep=sweep_configuration)
wandb.agent(sweep_id, function=train_func, count=20)

Create sweep with ID: 8e17pj90
Sweep URL: https://wandb.ai/japan-demo/jp-instruction-tuning/sweeps/8e17pj90


wandb: Agent Starting Run: g63ga8cb with config:
wandb: 	learning_rate: 0.001107952583922356
wandb: 	lora_alpha: 2
wandb: 	r: 4
wandb: Currently logged in as: keisuke-kamata (japan-demo). Use `wandb login --relogin` to force relogin


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).


trainable params: 1,310,720 || all params: 2,786,350,080 || trainable%: 0.04704075088798605


/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


...

trainable params: 2,621,440 || all params: 2,787,660,800 || trainable%: 0.09403726593995941


/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
10,2.926100,2.679382
20,2.737500,2.592502
30,2.613700,2.573694
40,2.617700,2.551992
50,2.611000,2.540586
60,2.386500,2.533543
70,2.728900,2.529042
80,2.622800,2.525728
90,2.648300,2.523227
100,2.560900,2.522573


wandb: Adding directory to artifact (./outputs/checkpoint-25)... Done. 0.0s
/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
wandb: Adding directory to artifact (./outputs/checkpoint-50)... Done. 0.0s
/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
wandb: Adding directory to artifact (./outputs/checkpoint-75)... Done. 0.0s
/opt/conda/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs

eval/loss,█▄▃▂▂▁▁▁▁▁
eval/runtime,██▆█▇▆▅▆▁█
eval/samples_per_second,▁▁▃▁▂▃▄▃█▁
eval/steps_per_second,▁▁▃▂▂▄▄▃█▂
train/epoch,▁▁▂▂▃▃▃▃▅▅▅▅▆▆▇▇▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,█▆▄▄▄▁▅▄▄▃
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
